In [1]:
# import libraries
#import boto3, re, sys, math, json, os, sagemaker, 
import urllib.request
#from sagemaker import get_execution_role
import numpy as np                                
import pandas as pd  
import matplotlib
import matplotlib.pyplot as plt                   
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime                 
#from sagemaker.predictor import csv_serializer   

In [77]:
# Grab the necessary dataset files from the origin sources
data_dir="/home/will/Desktop/CSC8515/"
unsw_nb15_mainurl = 'https://www.unsw.adfa.edu.au/unsw-canberra-cyber/cybersecurity/ADFA-NB15-Datasets/'
unsw_nb15_training_file = 'UNSW_NB15_training-set.csv'
unsw_nb15_feature_file = 'NUSW-NB15_features.csv' # misspelling in the origin data file name
unsw_nb15_test_file = 'UNSW_NB15_testing-set.csv'
download_training_url = unsw_nb15_mainurl + unsw_nb15_training_file
download_feature_url = unsw_nb15_mainurl + unsw_nb15_feature_file
download_test_url = unsw_nb15_mainurl + unsw_nb15_test_file


try:
  # urllib.request.urlretrieve (download_feature_url, unsw_nb15_feature_file)
  print('Success: ' + unsw_nb15_feature_file)
except Exception as e:
  print('Data load error: ',e)

try:
  # urllib.request.urlretrieve (download_training_url, unsw_nb15_training_file)
  print('Success: ' + unsw_nb15_training_file)
except Exception as e:
  print('Data load error: ',e)


Success: NUSW-NB15_features.csv
Success: UNSW_NB15_training-set.csv


In [79]:
# Load data
# Must declare data_dir as the directory of training and test files

raw_data_filename = data_dir + unsw_nb15_training_file
print('Loading raw data')
try:
    #raw_data = pd.read_csv(raw_data_filename, index_col=0, header=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)
raw_data.head()


Loading raw data
Success: Data loaded into dataframe.


,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
id,,,,,,,,,,,,,,,,,,,,,
1,0.000011,udp,-,INT,2,0,496,0,90909.0902,254,...,1,2,0,0,0,1,2,0,Normal,0
2,0.000008,udp,-,INT,2,0,1762,0,125000.0003,254,...,1,2,0,0,0,1,2,0,Normal,0
3,0.000005,udp,-,INT,2,0,1068,0,200000.0051,254,...,1,3,0,0,0,1,3,0,Normal,0
4,0.000006,udp,-,INT,2,0,900,0,166666.6608,254,...,1,3,0,0,0,2,3,0,Normal,0
5,0.000010,udp,-,INT,2,0,2126,0,100000.0025,254,...,1,3,0,0,0,2,3,0,Normal,0


In [81]:
# Now gather the list of feature names
print('Loading feature list')
try:
    feature_data = pd.read_csv(raw_data_filename, index_col=0, header=None, nrows=1)
    print('Success: Feature list loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)
#print(feature_data)
feature_list = feature_data.values
feature_list

Loading feature list
Success: Feature list loaded into dataframe.


array([['dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes',
        'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss',
        'dloss', 'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb',
        'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat', 'smean', 'dmean',
        'trans_depth', 'response_body_len', 'ct_srv_src', 'ct_state_ttl',
        'ct_dst_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm',
        'ct_dst_src_ltm', 'is_ftp_login', 'ct_ftp_cmd',
        'ct_flw_http_mthd', 'ct_src_ltm', 'ct_srv_dst',
        'is_sm_ips_ports', 'attack_cat', 'label']], dtype=object)

In [82]:

# Get even smaller dataset for experimentation
# remaining_data, selected_data = np.split(raw_data.sample(frac=1, random_state=1729), [int(0.80 * len(raw_data))])
print(remaining_data.shape, selected_data.shape)
selected_data.head(10)


(65865, 44) (16467, 44)


,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
id,,,,,,,,,,,,,,,,,,,,,
51467,3.010920,tcp,-,FIN,38,152,1884,180926,62.771511,62,...,1,1,0,0,0,1,1,0,Exploits,1
47425,1.303465,tcp,-,FIN,66,14,77491,612,60.607688,254,...,1,1,0,0,0,1,3,0,Fuzzers,1
60913,0.000010,udp,dns,INT,2,0,114,0,100000.002500,254,...,3,3,0,0,0,6,5,0,Generic,1
22022,0.000009,udp,dns,INT,2,0,114,0,111111.107200,254,...,9,9,0,0,0,9,9,0,Generic,1
43274,0.930510,tcp,-,FIN,30,14,25980,764,46.211218,254,...,1,6,0,0,0,1,5,0,Normal,0
19930,0.000007,udp,dns,INT,2,0,114,0,142857.140900,254,...,23,23,0,0,0,25,23,0,Generic,1
44330,1.537058,tcp,http,FIN,18,56,1182,69091,47.493328,62,...,1,1,0,0,0,1,1,0,Exploits,1
4219,0.000006,unas,-,INT,2,0,200,0,166666.660800,254,...,3,10,0,0,0,12,9,0,Exploits,1
44263,0.388297,tcp,http,FIN,10,8,630,354,43.780921,254,...,1,3,0,0,1,3,11,0,Fuzzers,1


In [83]:
# Write the samll data set to disk
# Read back the small data
samll_filename = data_dir + 'UNSW_NB15_training-small.csv'

# selected_data.to_csv(samll_filename)


try:
    # small_data = pd.read_csv(samll_filename, index_col=0, header=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)
print(small_data.shape)
selected_data = small_data
print(selected_data.shape)
selected_data.head()


Success: Data loaded into dataframe.
(16467, 44)
(16467, 44)


,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
id,,,,,,,,,,,,,,,,,,,,,
51467,3.010920,tcp,-,FIN,38,152,1884,180926,62.771511,62,...,1,1,0,0,0,1,1,0,Exploits,1
47425,1.303465,tcp,-,FIN,66,14,77491,612,60.607688,254,...,1,1,0,0,0,1,3,0,Fuzzers,1
60913,0.000010,udp,dns,INT,2,0,114,0,100000.002500,254,...,3,3,0,0,0,6,5,0,Generic,1
22022,0.000009,udp,dns,INT,2,0,114,0,111111.107200,254,...,9,9,0,0,0,9,9,0,Generic,1
43274,0.930510,tcp,-,FIN,30,14,25980,764,46.211218,254,...,1,6,0,0,0,1,5,0,Normal,0


In [84]:
print('Transforming data')
# Factorize columns: "proto", "service", "state", "attack_cat"
selected_data['proto'], protocols = pd.factorize(selected_data['proto'])
selected_data['service'], services = pd.factorize(selected_data['service'])
selected_data['state'], states    = pd.factorize(selected_data['state'])
#selected_data['attack_cat'], attacks = pd.factorize(selected_data['attack_cat'])
selected_data.head()

Transforming data


,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
id,,,,,,,,,,,,,,,,,,,,,
51467,3.010920,0,0,0,38,152,1884,180926,62.771511,62,...,1,1,0,0,0,1,1,0,Exploits,1
47425,1.303465,0,0,0,66,14,77491,612,60.607688,254,...,1,1,0,0,0,1,3,0,Fuzzers,1
60913,0.000010,1,1,1,2,0,114,0,100000.002500,254,...,3,3,0,0,0,6,5,0,Generic,1
22022,0.000009,1,1,1,2,0,114,0,111111.107200,254,...,9,9,0,0,0,9,9,0,Generic,1
43274,0.930510,0,0,0,30,14,25980,764,46.211218,254,...,1,6,0,0,0,1,5,0,Normal,0


In [173]:
from sklearn import datasets
from sklearn import neighbors
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [174]:
# separate features (columns 1..42) and labels (column 43, 44)
features= selected_data.iloc[:,:selected_data.shape[1]-2]
#labels= selected_data.iloc[:,selected_data.shape[1]-1:]
attack_labels=selected_data['attack_cat']
print('features shape is:', features.shape, 'labels shape is:', attack_labels.shape)
attack_labels.head(10)


features shape is: (16467, 42) labels shape is: (16467,)


id
51467    Exploits
47425     Fuzzers
60913     Generic
22022     Generic
43274      Normal
19930     Generic
44330    Exploits
4219     Exploits
44263     Fuzzers
1355     Exploits
Name: attack_cat, dtype: object

In [175]:
anormaly_labels=selected_data['label']
anormaly_labels.head(10)

id
51467    1
47425    1
60913    1
22022    1
43274    0
19930    1
44330    1
4219     1
44263    1
1355     1
Name: label, dtype: int64

In [197]:
# Study the training data
labels= attack_labels.values # this becomes a 'horizontal' array
#print(labels)
UNSW_NB15_Label_names2 = np.unique(labels, return_counts=True)
#print(UNSW_NB15_Label_names2)
UNSW_NB15_Label_names = {}
for i in range(len(UNSW_NB15_Label_names2[0])):
    UNSW_NB15_Label_names[UNSW_NB15_Label_names2[0][i]] = UNSW_NB15_Label_names2[1][i]
# ordered_UNSW_NB15_Label_names = sorted(UNSW_NB15_Label_names.items(), key=lambda x: x[1], reverse=True)
ordered_UNSW_NB15_Label_names = sorted(UNSW_NB15_Label_names, key=UNSW_NB15_Label_names.__getitem__, reverse=True)
#UNSW_NB15_Label_names = np.sort(UNSW_NB15_Label_names, axis=0, order=Count)
print('Attack Category', '\t\t\t Counts')
short_cat_name = np.copy(ordered_UNSW_NB15_Label_names)
for i, cat in enumerate(short_cat_name):
    if(cat == 'Reconnaissance'):
        short_cat_name[i] = 'Reconn'
for i, cat in enumerate(short_cat_name):
    if(len(cat) < 6):
        print('\t', cat, '\t\t\t\t', UNSW_NB15_Label_names[ordered_UNSW_NB15_Label_names[i]])
    else:
        print('\t', cat, '\t\t\t', UNSW_NB15_Label_names[ordered_UNSW_NB15_Label_names[i]])
print('Total # of Data Points: \t\t', attack_labels.shape[0])

    

Attack Category 			 Counts
	 Normal 			 7286
	 Generic 			 3821
	 Exploits 			 2282
	 Fuzzers 			 1210
	 DoS 				 837
	 Reconn 			 690
	 Analysis 			 141
	 Backdoor 			 113
	 Shellcode 			 77
	 Worms 				 10
Total # of Data Points: 		 16467


In [179]:
# Separate data in train set and test set
df= pd.DataFrame(features)
# create training and testing vars
# Note: train_size + test_size < 1.0 means we are subsampling
# Use small numbers for slow classifiers, as KNN, Radius, SVC,...
X_train, X_test, y_train, y_test = train_test_split(df, labels, train_size=0.8, test_size=0.2, random_state=1)
print('X_train, y_train:', X_train.shape, y_train.shape)
print('X_test, y_test:', X_test.shape, y_test.shape)
X_train.describe()

X_train, y_train: (13173, 42) (13173,)
X_test, y_test: (3294, 42) (3294,)


,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports
count,13173.000000,13173.000000,13173.000000,13173.000000,13173.000000,13173.000000,1.317300e+04,1.317300e+04,1.317300e+04,13173.000000,...,13173.000000,13173.000000,13173.000000,13173.000000,13173.000000,13173.000000,13173.000000,13173.000000,13173.000000,13173.000000
mean,1.040637,4.005314,0.749867,0.649662,19.440902,18.205724,8.456119e+03,1.417231e+04,8.410443e+04,181.707356,...,5.815304,4.997267,3.723222,7.569802,0.008047,0.008123,0.130494,6.556138,9.283762,0.011007
std,4.817843,16.149289,1.227804,0.719147,139.116070,126.833104,1.777884e+05,1.666998e+05,1.489297e+05,101.409419,...,8.485038,8.477780,5.973248,11.549801,0.090191,0.091438,0.617507,8.620124,11.235563,0.104341
min,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,2.400000e+01,0.000000e+00,0.000000e+00,0.000000,...,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
25%,0.000008,0.000000,0.000000,0.000000,2.000000,0.000000,1.140000e+02,0.000000e+00,2.878919e+01,62.000000,...,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,2.000000,0.000000
50%,0.012917,0.000000,0.000000,1.000000,6.000000,2.000000,5.340000e+02,1.780000e+02,2.757353e+03,254.000000,...,2.000000,1.000000,1.000000,3.000000,0.000000,0.000000,0.000000,3.000000,5.000000,0.000000
75%,0.714544,1.000000,1.000000,1.000000,12.000000,10.000000,1.276000e+03,8.500000e+02,1.250000e+05,254.000000,...,6.000000,4.000000,3.000000,6.000000,0.000000,0.000000,0.000000,7.000000,11.000000,0.000000
max,59.996918,130.000000,11.000000,3.000000,10200.000000,10970.000000,1.367739e+07,1.465542e+07,1.000000e+06,255.000000,...,59.000000,59.000000,37.000000,63.000000,2.000000,2.000000,16.000000,60.000000,59.000000,1.000000


In [180]:
#remove redundant features
#train_data['num_outbound_cmds'].value_counts()
#train_data.drop('num_outbound_cmds', axis=1, inplace=True)
#train_data['is_host_login'].value_counts()
#train_data.drop('is_host_login', axis=1, inplace=True)
#train_data.head(10)

In [202]:
# Training, choose model by commenting/uncommenting clf=
print('Training model...')
# clf= RandomForestClassifier(n_jobs=-1, random_state=3, n_estimators=102)
#, max_features=0.8, min_samples_leaf=3, n_estimators=500, min_samples_split=3, random_state=10, verbose=1)
# clf = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, class_weight=None, presort=False)
clf= svm.SVC(kernel='rbf')
#clf= neighbors.KNeighborsClassifier(n_neighbors=1, algorithm='ball_tree', metric='manhattan')

trained_model= clf.fit(X_train, y_train)


Training model...


In [204]:
# Predicting
print('Predicting...')
y_pred = clf.predict(X_test)
#print('Predited labels: \t', y_pred)
#print('True labels: \t\t', y_test)

print('Computing performance metrics')
#results = confusion_matrix(y_test, y_pred)
results = confusion_matrix(y_test, y_pred, short_cat_name)
print('Confusion matrix:\n', results)

print("Classificationreport:")
print(classification_report(y_test,y_pred,target_names=short_cat_name))
#print(classification_report(y_test,y_pred))


Predicting...
Computing performance metrics
Confusion matrix:
 [[1435    0    1    1    0    0    0    0    0    0]
 [  64  712    2    0    2    0    0    0    0    0]
 [ 325    0   74   13   34    0    3    0    0    0]
 [ 191    0   26   24    7    0    2    1    0    0]
 [  62    0   77    7   36    0    2    1    0    1]
 [   0    0    0    0    0    0    0    0    0    0]
 [   5    0   13    8    4    0    1    0    0    0]
 [   6    0   11    3    2    0    0    0    0    0]
 [  15    0    0    0    0    0    0    0    0    0]
 [   2    0    0    0    0    0    0    0    0    0]]
Classificationreport:
             precision    recall  f1-score   support

     Normal       0.12      0.03      0.05        31
    Generic       0.00      0.00      0.00        22
   Exploits       0.39      0.19      0.26       187
    Fuzzers       0.35      0.16      0.22       450
        DoS       0.43      0.10      0.16       251
     Reconn       1.00      0.91      0.95       780
   Analysis 

/home/will/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [205]:
# Function to plot confusion matrix
# https://gist.github.com/zachguo/10296432
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    empty_cell0 = " " * (columnwidth-1) + "-"
    
    # Begin CHANGES
    fst_empty_cell = (columnwidth-3)//2 * " " + "t/p" + (columnwidth-3)//2 * " "
    
    if len(fst_empty_cell) < len(empty_cell):
        fst_empty_cell = " " * (len(empty_cell) - len(fst_empty_cell)) + fst_empty_cell
    # Print header
    print("    " + fst_empty_cell, end=" ")
    # End CHANGES
    
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
        
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell0
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()
        

In [206]:
print_cm(results, short_cat_name, True)

       t/p       Normal   Generic  Exploits   Fuzzers       DoS    Reconn  Analysis  Backdoor Shellcode     Worms 
       Normal    1435.0         -       1.0       1.0         -         -         -         -         -         - 
      Generic      64.0     712.0       2.0         -       2.0         -         -         -         -         - 
     Exploits     325.0         -      74.0      13.0      34.0         -       3.0         -         -         - 
      Fuzzers     191.0         -      26.0      24.0       7.0         -       2.0       1.0         -         - 
          DoS      62.0         -      77.0       7.0      36.0         -       2.0       1.0         -       1.0 
       Reconn         -         -         -         -         -         -         -         -         -         - 
     Analysis       5.0         -      13.0       8.0       4.0         -       1.0         -         -         - 
     Backdoor       6.0         -      11.0       3.0       2.0         -       